## Load latest headlines

In [13]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-05-19-38-00 +0000,wsj,Stock Market Today: Trump-Musk Feud Hits Tesla,https://www.wsj.com/livecoverage/stock-market-...
1,2025-06-05-19-38-00 +0000,wsj,Opinion | Notable & Quotable: Democrats and Yo...,https://www.wsj.com/opinion/notable-quotable-d...
2,2025-06-05-19-35-00 +0000,wsj,Early hints of a slowing labor market are rais...,https://www.wsj.com/articles/labor-market-outl...
3,2025-06-05-19-30-20 +0000,nyt,Live Video: Ispace of Japan’s Resilience Lande...,https://www.nytimes.com/2025/06/05/science/moo...
4,2025-06-05-19-25-42 +0000,nyt,Trump’s Travel Ban Could Shake Up Internationa...,https://www.nytimes.com/2025/06/05/world/ameri...


## Calculate word frequencies

In [14]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_38396\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,28
126,new,8
43,up,6
40,travel,6
200,president,5


## Rank headlines by score

In [15]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
36,2025-06-05-13-43-00 +0000,wsj,President Trump has signed a sweeping travel b...,https://www.wsj.com/politics/policy/trump-trav...,58
41,2025-06-05-12-30-20 +0000,nyt,"In War-Torn Nations, Trump’s Travel Ban Brings...",https://www.nytimes.com/2025/06/05/world/asia/...,55
56,2025-06-05-03-12-00 +0000,wsj,President Trump ordered an investigation into ...,https://www.wsj.com/politics/policy/trump-orde...,53
29,2025-06-05-15-14-00 +0000,wsj,The ECB cut rates for the eighth time in a yea...,https://www.wsj.com/economy/central-banking/ec...,50
71,2025-06-04-22-18-00 +0000,wsj,President Trump is losing patience with Elon M...,https://www.wsj.com/politics/policy/elon-musk-...,49


## Select top headlines

In [16]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
36,58,2025-06-05-13-43-00 +0000,wsj,President Trump has signed a sweeping travel b...,https://www.wsj.com/politics/policy/trump-trav...
2,31,2025-06-05-19-35-00 +0000,wsj,Early hints of a slowing labor market are rais...,https://www.wsj.com/articles/labor-market-outl...
54,25,2025-06-05-03-49-26 +0000,nyt,Drone Attacks Are the New Front in War. Can NA...,https://www.nytimes.com/2025/06/04/world/europ...
32,25,2025-06-05-14-59-00 +0000,wsj,Violence against Jews is about more than left ...,https://www.wsj.com/politics/violence-against-...
43,24,2025-06-05-10-38-00 +0000,wsj,Israeli Forces Recover Bodies of American-Isra...,https://www.wsj.com/world/middle-east/israeli-...
66,19,2025-06-04-23-37-00 +0000,wsj,A federal judge ruled Wednesday that the Trump...,https://www.wsj.com/politics/policy/migrants-s...
9,18,2025-06-05-18-55-00 +0000,wsj,Long-simmering tensions between Trump and Elon...,https://www.wsj.com/politics/policy/trump-rips...
29,17,2025-06-05-15-14-00 +0000,wsj,The ECB cut rates for the eighth time in a yea...,https://www.wsj.com/economy/central-banking/ec...
35,17,2025-06-05-14-47-00 +0000,wsj,"The U.S. trade deficit collapsed in April, as ...",https://www.wsj.com/economy/trade/u-s-trade-de...
67,16,2025-06-04-22-41-13 +0000,nyt,Trump and Putin Discuss Ukraine Drone Strike a...,https://www.nytimes.com/2025/06/04/us/politics...


## Next steps